In [164]:
import pandas as pd
import numpy as np

In [165]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/weatherAUS.csv')
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,ENE,13.0,11.0,51.0,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,No
145456,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,N,13.0,9.0,56.0,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,No
145457,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,WNW,9.0,9.0,53.0,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,No
145458,2017-06-24,Uluru,7.8,27.0,0.0,NaN,NaN,SE,28.0,SSE,N,13.0,7.0,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,No,No


In [166]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [167]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [168]:
df.shape

(145460, 23)

In [169]:
df.isnull().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

In [170]:
df.RainTomorrow.value_counts()

No     110316
Yes     31877
Name: RainTomorrow, dtype: int64

In [171]:
# Perform undersampling to balance dataset
num_Yes = len(df[df['RainTomorrow'] == 'Yes'])
num_No = len(df[df['RainTomorrow'] == 'No'])
Yes_indices = df[df.RainTomorrow == 'Yes'].index
No_indices = df[df.RainTomorrow == 'No'].index
random_indices = np.random.choice(No_indices,num_Yes, replace=False)
under_sample_indices = np.concatenate([Yes_indices,random_indices])
under_sample = df.loc[under_sample_indices]
print(under_sample.groupby(['RainTomorrow']).size())

RainTomorrow
No     31877
Yes    31877
dtype: int64


In [172]:
under_sample.isnull().sum()

Date                 0
Location             0
MinTemp            317
MaxTemp            143
Rainfall           891
Evaporation      27457
Sunshine         30327
WindGustDir       4474
WindGustSpeed     4437
WindDir9am        4313
WindDir3pm        1857
WindSpeed9am       652
WindSpeed3pm      1279
Humidity9am        901
Humidity3pm       1756
Pressure9am       6337
Pressure3pm       6338
Cloud9am         23456
Cloud3pm         24797
Temp9am            489
Temp3pm           1353
RainToday          891
RainTomorrow         0
dtype: int64

In [173]:
under_sample

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
8,2008-12-09,Albury,9.7,31.9,0.0,NaN,NaN,NNW,80.0,SE,NW,7.0,28.0,42.0,9.0,1008.9,1003.6,NaN,NaN,18.3,30.2,No,Yes
10,2008-12-11,Albury,13.4,30.4,0.0,NaN,NaN,N,30.0,SSE,ESE,17.0,6.0,48.0,22.0,1011.8,1008.7,NaN,NaN,20.4,28.8,No,Yes
11,2008-12-12,Albury,15.9,21.7,2.2,NaN,NaN,NNE,31.0,NE,ENE,15.0,13.0,89.0,91.0,1010.5,1004.2,8.0,8.0,15.9,17.0,Yes,Yes
12,2008-12-13,Albury,15.9,18.6,15.6,NaN,NaN,W,61.0,NNW,NNW,28.0,28.0,76.0,93.0,994.3,993.0,8.0,8.0,17.4,15.8,Yes,Yes
16,2008-12-17,Albury,14.1,20.9,0.0,NaN,NaN,ENE,22.0,SSW,E,11.0,9.0,69.0,82.0,1012.2,1010.4,8.0,1.0,17.2,18.1,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89275,2014-11-04,Cairns,23.8,31.8,0.0,8.2,5.6,ENE,35.0,SE,ESE,13.0,26.0,60.0,52.0,1017.3,1013.4,7.0,7.0,28.1,29.0,No,No
108681,2009-03-21,Albany,12.6,19.5,4.6,3.4,3.0,NaN,NaN,NW,SSW,6.0,7.0,83.0,59.0,1016.6,1016.2,6.0,6.0,14.5,17.4,Yes,No
95617,2015-07-24,Townsville,21.6,27.0,0.0,5.6,5.3,ENE,30.0,ESE,NE,15.0,24.0,77.0,67.0,1020.9,1017.9,7.0,2.0,23.6,25.9,No,No
2055,2014-10-15,Albury,2.9,19.5,0.2,NaN,NaN,SE,22.0,W,N,4.0,2.0,73.0,42.0,1018.9,1016.2,NaN,1.0,9.7,16.6,No,No


In [174]:
# Fill missing values
categorical_features = under_sample[['WindGustDir','WindDir9am','WindDir3pm','RainToday','RainTomorrow']]
numerical_features = under_sample.drop(['WindGustDir','WindDir9am','WindDir3pm','RainToday','RainTomorrow','Date','Location'], axis = 1)

numerical_imputer = SimpleImputer(strategy='mean')
categorical_imputer = SimpleImputer(strategy='most_frequent')

complete_numerical = numerical_imputer.fit_transform(numerical_features)
complete_categorical = categorical_imputer.fit_transform(categorical_features)

In [175]:
df2=pd.DataFrame(complete_numerical)
df2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,9.7,31.9,0.0,5.190272,6.49694,80.000000,7.0,28.0,42.0,9.0,1008.9,1003.6,5.035486,5.186231,18.3,30.2
1,13.4,30.4,0.0,5.190272,6.49694,30.000000,17.0,6.0,48.0,22.0,1011.8,1008.7,5.035486,5.186231,20.4,28.8
2,15.9,21.7,2.2,5.190272,6.49694,31.000000,15.0,13.0,89.0,91.0,1010.5,1004.2,8.000000,8.000000,15.9,17.0
3,15.9,18.6,15.6,5.190272,6.49694,61.000000,28.0,28.0,76.0,93.0,994.3,993.0,8.000000,8.000000,17.4,15.8
4,14.1,20.9,0.0,5.190272,6.49694,22.000000,11.0,9.0,69.0,82.0,1012.2,1010.4,8.000000,1.000000,17.2,18.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63749,23.8,31.8,0.0,8.200000,5.60000,35.000000,13.0,26.0,60.0,52.0,1017.3,1013.4,7.000000,7.000000,28.1,29.0
63750,12.6,19.5,4.6,3.400000,3.00000,42.074228,6.0,7.0,83.0,59.0,1016.6,1016.2,6.000000,6.000000,14.5,17.4
63751,21.6,27.0,0.0,5.600000,5.30000,30.000000,15.0,24.0,77.0,67.0,1020.9,1017.9,7.000000,2.000000,23.6,25.9
63752,2.9,19.5,0.2,5.190272,6.49694,22.000000,4.0,2.0,73.0,42.0,1018.9,1016.2,5.035486,1.000000,9.7,16.6


In [176]:
df2.columns=['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustSpeed','WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm']
df2

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
0,9.7,31.9,0.0,5.190272,6.49694,80.000000,7.0,28.0,42.0,9.0,1008.9,1003.6,5.035486,5.186231,18.3,30.2
1,13.4,30.4,0.0,5.190272,6.49694,30.000000,17.0,6.0,48.0,22.0,1011.8,1008.7,5.035486,5.186231,20.4,28.8
2,15.9,21.7,2.2,5.190272,6.49694,31.000000,15.0,13.0,89.0,91.0,1010.5,1004.2,8.000000,8.000000,15.9,17.0
3,15.9,18.6,15.6,5.190272,6.49694,61.000000,28.0,28.0,76.0,93.0,994.3,993.0,8.000000,8.000000,17.4,15.8
4,14.1,20.9,0.0,5.190272,6.49694,22.000000,11.0,9.0,69.0,82.0,1012.2,1010.4,8.000000,1.000000,17.2,18.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63749,23.8,31.8,0.0,8.200000,5.60000,35.000000,13.0,26.0,60.0,52.0,1017.3,1013.4,7.000000,7.000000,28.1,29.0
63750,12.6,19.5,4.6,3.400000,3.00000,42.074228,6.0,7.0,83.0,59.0,1016.6,1016.2,6.000000,6.000000,14.5,17.4
63751,21.6,27.0,0.0,5.600000,5.30000,30.000000,15.0,24.0,77.0,67.0,1020.9,1017.9,7.000000,2.000000,23.6,25.9
63752,2.9,19.5,0.2,5.190272,6.49694,22.000000,4.0,2.0,73.0,42.0,1018.9,1016.2,5.035486,1.000000,9.7,16.6


In [177]:
df3=pd.DataFrame(complete_categorical)
df3

,0,1,2,3,4
0,NNW,SE,NW,No,Yes
1,N,SSE,ESE,No,Yes
2,NNE,NE,ENE,Yes,Yes
3,W,NNW,NNW,Yes,Yes
4,ENE,SSW,E,No,Yes
...,...,...,...,...,...
63749,ENE,SE,ESE,No,No
63750,W,NW,SSW,Yes,No
63751,ENE,ESE,NE,No,No
63752,SE,W,N,No,No


In [178]:
df3.columns=['WindGustDir','WindDir9am','WindDir3pm','RainToday','RainTomorrow']
df3

,WindGustDir,WindDir9am,WindDir3pm,RainToday,RainTomorrow
0,NNW,SE,NW,No,Yes
1,N,SSE,ESE,No,Yes
2,NNE,NE,ENE,Yes,Yes
3,W,NNW,NNW,Yes,Yes
4,ENE,SSW,E,No,Yes
...,...,...,...,...,...
63749,ENE,SE,ESE,No,No
63750,W,NW,SSW,Yes,No
63751,ENE,ESE,NE,No,No
63752,SE,W,N,No,No


In [179]:
df3["Location"]=list(under_sample["Location"])
df3

,WindGustDir,WindDir9am,WindDir3pm,RainToday,RainTomorrow,Location
0,NNW,SE,NW,No,Yes,Albury
1,N,SSE,ESE,No,Yes,Albury
2,NNE,NE,ENE,Yes,Yes,Albury
3,W,NNW,NNW,Yes,Yes,Albury
4,ENE,SSW,E,No,Yes,Albury
...,...,...,...,...,...,...
63749,ENE,SE,ESE,No,No,Cairns
63750,W,NW,SSW,Yes,No,Albany
63751,ENE,ESE,NE,No,No,Townsville
63752,SE,W,N,No,No,Albury


In [180]:
dummy_cat=pd.get_dummies(df3,drop_first=True)
dummy_cat

,WindGustDir_ENE,WindGustDir_ESE,WindGustDir_N,WindGustDir_NE,WindGustDir_NNE,WindGustDir_NNW,WindGustDir_NW,WindGustDir_S,WindGustDir_SE,WindGustDir_SSE,WindGustDir_SSW,WindGustDir_SW,WindGustDir_W,WindGustDir_WNW,WindGustDir_WSW,WindDir9am_ENE,WindDir9am_ESE,WindDir9am_N,WindDir9am_NE,WindDir9am_NNE,WindDir9am_NNW,WindDir9am_NW,WindDir9am_S,WindDir9am_SE,WindDir9am_SSE,WindDir9am_SSW,WindDir9am_SW,WindDir9am_W,WindDir9am_WNW,WindDir9am_WSW,WindDir3pm_ENE,WindDir3pm_ESE,WindDir3pm_N,WindDir3pm_NE,WindDir3pm_NNE,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,...,Location_Canberra,Location_Cobar,Location_CoffsHarbour,Location_Dartmoor,Location_Darwin,Location_GoldCoast,Location_Hobart,Location_Katherine,Location_Launceston,Location_Melbourne,Location_MelbourneAirport,Location_Mildura,Location_Moree,Location_MountGambier,Location_MountGinini,Location_Newcastle,Location_Nhil,Location_NorahHead,Location_NorfolkIsland,Location_Nuriootpa,Location_PearceRAAF,Location_Penrith,Location_Perth,Location_PerthAirport,Location_Portland,Location_Richmond,Location_Sale,Location_SalmonGums,Location_Sydney,Location_SydneyAirport,Location_Townsville,Location_Tuggeranong,Location_Uluru,Location_WaggaWagga,Location_Walpole,Location_Watsonia,Location_Williamtown,Location_Witchcliffe,Location_Wollongong,Location_Woomera
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63749,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
63750,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
63751,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
63752,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [181]:
complete_data = pd.concat([df2,dummy_cat], axis=1)

In [182]:
complete_data

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,WindGustDir_ENE,WindGustDir_ESE,WindGustDir_N,WindGustDir_NE,WindGustDir_NNE,WindGustDir_NNW,WindGustDir_NW,WindGustDir_S,WindGustDir_SE,WindGustDir_SSE,WindGustDir_SSW,WindGustDir_SW,WindGustDir_W,WindGustDir_WNW,WindGustDir_WSW,WindDir9am_ENE,WindDir9am_ESE,WindDir9am_N,WindDir9am_NE,WindDir9am_NNE,WindDir9am_NNW,WindDir9am_NW,WindDir9am_S,WindDir9am_SE,...,Location_Canberra,Location_Cobar,Location_CoffsHarbour,Location_Dartmoor,Location_Darwin,Location_GoldCoast,Location_Hobart,Location_Katherine,Location_Launceston,Location_Melbourne,Location_MelbourneAirport,Location_Mildura,Location_Moree,Location_MountGambier,Location_MountGinini,Location_Newcastle,Location_Nhil,Location_NorahHead,Location_NorfolkIsland,Location_Nuriootpa,Location_PearceRAAF,Location_Penrith,Location_Perth,Location_PerthAirport,Location_Portland,Location_Richmond,Location_Sale,Location_SalmonGums,Location_Sydney,Location_SydneyAirport,Location_Townsville,Location_Tuggeranong,Location_Uluru,Location_WaggaWagga,Location_Walpole,Location_Watsonia,Location_Williamtown,Location_Witchcliffe,Location_Wollongong,Location_Woomera
0,9.7,31.9,0.0,5.190272,6.49694,80.000000,7.0,28.0,42.0,9.0,1008.9,1003.6,5.035486,5.186231,18.3,30.2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,13.4,30.4,0.0,5.190272,6.49694,30.000000,17.0,6.0,48.0,22.0,1011.8,1008.7,5.035486,5.186231,20.4,28.8,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,15.9,21.7,2.2,5.190272,6.49694,31.000000,15.0,13.0,89.0,91.0,1010.5,1004.2,8.000000,8.000000,15.9,17.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,15.9,18.6,15.6,5.190272,6.49694,61.000000,28.0,28.0,76.0,93.0,994.3,993.0,8.000000,8.000000,17.4,15.8,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,14.1,20.9,0.0,5.190272,6.49694,22.000000,11.0,9.0,69.0,82.0,1012.2,1010.4,8.000000,1.000000,17.2,18.1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63749,23.8,31.8,0.0,8.200000,5.60000,35.000000,13.0,26.0,60.0,52.0,1017.3,1013.4,7.000000,7.000000,28.1,29.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
63750,12.6,19.5,4.6,3.400000,3.00000,42.074228,6.0,7.0,83.0,59.0,1016.6,1016.2,6.000000,6.000000,14.5,17.4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
63751,21.6,27.0,0.0,5.600000,5.30000,30.000000,15.0,24.0,77.0,67.0,1020.9,1017.9,7.000000,2.000000,23.6,25.9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
63752,2.9,19.5,0.2,5.190272,6.49694,22.000000,4.0,2.0,73.0,42.0,1018.9,1016.2,5.035486,1.000000,9.7,16.6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [183]:
X = complete_data.drop(columns=["RainTomorrow_Yes"])
y = complete_data.RainTomorrow_Yes

In [184]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [185]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [186]:
#Use logistic regression to train the model
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train, y_train)

LogisticRegression()

In [187]:
predictions = logmodel.predict(X_test)
predictions

array([0, 1, 0, ..., 0, 0, 1], dtype=uint8)

In [188]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.7873163590735609

In [189]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[7664, 1934],
       [2134, 7395]])

In [190]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 20)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [191]:
logmodel_PCA = LogisticRegression(random_state=0)
logmodel_PCA.fit(X_train_pca, y_train)

LogisticRegression(random_state=0)

In [192]:
predictions_PCA = logmodel_PCA.predict(X_test_pca)

In [193]:
accuracy_score(y_test, predictions_PCA)

0.7569927327861139

In [194]:
confusion_matrix(y_test, predictions_PCA)

array([[7259, 2339],
       [2309, 7220]])

In [195]:
from sklearn.metrics import classification_report
print(f"Report\n {classification_report(y_test,predictions_PCA)}")

Report
               precision    recall  f1-score   support

           0       0.76      0.76      0.76      9598
           1       0.76      0.76      0.76      9529

    accuracy                           0.76     19127
   macro avg       0.76      0.76      0.76     19127
weighted avg       0.76      0.76      0.76     19127



In [151]:
#Use KNN Classifier to train the model
from sklearn.neighbors import KNeighborsClassifier

#import metrics model to check the accuracy 
from sklearn import metrics

#Try running from k=1 through 25 and record testing accuracy
k_range = range(1000,1001)
scores = {}
scores_list = []
for k in k_range:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train_pca, y_train)
        y_pred=knn.predict(X_test_pca)
        scores[k] = metrics.accuracy_score(y_test,y_pred)
        scores_list.append(metrics.accuracy_score(y_test,y_pred))

In [146]:
scores

{1: 0.6959454160458003,
 2: 0.6867696651243039,
 3: 0.7258254254568269,
 4: 0.7200219590620344,
 5: 0.7364128303662458,
 6: 0.733354246725747,
 7: 0.7473923613834209,
 8: 0.7450396047368834,
 9: 0.752411575562701,
 10: 0.750529370245471,
 11: 0.7559407105325072,
 12: 0.7528037016704572,
 13: 0.7587640185083523,
 14: 0.7588424437299035,
 15: 0.7623715786997098,
 16: 0.7607246490471335,
 17: 0.7636263822445298,
 18: 0.762450003921261,
 19: 0.7639400831307348}

In [148]:
scores

{20: 0.7638616579091836,
 21: 0.7652733118971061,
 22: 0.7640185083522861,
 23: 0.7650380362324524,
 24: 0.7648811857893498,
 25: 0.7667633911065799,
 26: 0.7660575641126186,
 27: 0.7676260685436437,
 28: 0.7675476433220924,
 29: 0.7679397694298487,
 30: 0.7678613442082974,
 31: 0.7673907928789899,
 32: 0.7687240216453611,
 33: 0.7699788251901811,
 34: 0.7695082738608736,
 35: 0.7696651243039762,
 36: 0.7681750450945024,
 37: 0.7687240216453611,
 38: 0.7684103207591562,
 39: 0.7678613442082974}

In [152]:
scores

{1000: 0.7576660654066347}

In [206]:
# Use Randomforest Classifier to the model
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=20)
classifier.fit(X_train_pca, y_train)

RandomForestClassifier(n_estimators=20)

In [207]:
y_pred = classifier.predict(X_test_pca)

In [208]:
confusion_matrix(y_test, y_pred)

array([[7474, 2124],
       [2473, 7056]])

In [209]:
print(f"Report\n {classification_report(y_test,y_pred)}")

Report
               precision    recall  f1-score   support

           0       0.75      0.78      0.76      9598
           1       0.77      0.74      0.75      9529

    accuracy                           0.76     19127
   macro avg       0.76      0.76      0.76     19127
weighted avg       0.76      0.76      0.76     19127



In [210]:
from sklearn.metrics import f1_score
f1_score(y_test,y_pred,average='macro')

0.7595442804096799

In [211]:
for i in [10,100,500]:
    classifier = RandomForestClassifier(n_estimators=i)
    classifier.fit(X_train_pca, y_train)
    y_pred = classifier.predict(X_test_pca)
    confusion_matrix(y_test, y_pred)
    k=f1_score(y_test,y_pred,average='macro')
    print("i:",i,"f1:",k)


i: 10 f1: 0.7472948554503247
i: 100 f1: 0.7720492184220056
i: 500 f1: 0.7741393987232158


In [212]:
for i in [1000,2000]:
    classifier = RandomForestClassifier(n_estimators=i)
    classifier.fit(X_train_pca, y_train)
    y_pred = classifier.predict(X_test_pca)
    confusion_matrix(y_test, y_pred)
    k=f1_score(y_test,y_pred,average='macro')
    print("i:",i,"f1:",k)

i: 1000 f1: 0.7746614850494005
i: 2000 f1: 0.7751839828456234


In [213]:
#Use LightGBM to train the model
from lightgbm import LGBMClassifier
model = LGBMClassifier()
model.fit(X_train_pca, y_train)

y_pred = model.predict(X_test_pca)
y_pred_proba = model.predict_proba(X_test_pca)

from sklearn.metrics import roc_auc_score
LightGBM_score = roc_auc_score(y_test, y_pred_proba[:,1])
LightGBM_score

0.853850681541094